In [1]:
#Question 1
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split

In [2]:
df = pd.read_csv('USA_Housing.csv')


In [33]:
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [5]:
X = df.drop("Price", axis=1).values
y = df["Price"].values.reshape(-1, 1)

In [13]:
scaler = StandardScaler()     
X_scaled = scaler.fit_transform(X)

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [17]:
best_beta = None
best_r2 = -np.inf    
r2_scores = []

In [19]:
for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Add bias column of ones for intercept
    X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

    # Compute beta using Least Squares: β = (XᵀX)^(-1) Xᵀy
    beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ (X_train_bias.T @ y_train)
     # Predictions
    y_pred = X_test_bias @ beta

    # R2 score
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"Fold {fold+1}: R2 Score = {r2:.4f}")

    # Track best beta
    if r2 > best_r2:
        best_r2 = r2
        best_beta = beta

Fold 1: R2 Score = 0.9180
Fold 2: R2 Score = 0.9146
Fold 3: R2 Score = 0.9116
Fold 4: R2 Score = 0.9193
Fold 5: R2 Score = 0.9244


In [21]:
print("\nAverage R2 Score across 5 folds:", np.mean(r2_scores))
print("Best R2 Score:", best_r2)


Average R2 Score across 5 folds: 0.9175745431092714
Best R2 Score: 0.9243869413350316


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [25]:
X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

In [27]:
y_train_pred = X_train_bias @ best_beta
y_test_pred = X_test_bias @ best_beta

In [29]:
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [31]:
print("\nFinal Model Performance using Best Beta:")
print(f"Train R2 Score: {train_r2:.4f}")
print(f"Test R2 Score: {test_r2:.4f}")


Final Model Performance using Best Beta:
Train R2 Score: 0.9193
Test R2 Score: 0.9147


In [37]:
print(beta)

[[1.23161736e+06]
 [2.30225051e+05]
 [1.63956839e+05]
 [1.21115120e+05]
 [7.83467170e+02]
 [1.50662447e+05]]


In [7]:
#Ques 2

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42)

print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

def gradient_descent(X, y, alpha, iterations=1000):
    n_samples, n_features = X.shape
    beta = np.zeros((n_features, 1))

    for _ in range(iterations):
        gradients = (1 / n_samples) * X.T @ (X @ beta - y)
        beta -= alpha * gradients
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
results = []

best_beta = None
best_val_r2 = -np.inf
best_lr = None

for lr in learning_rates:
    beta = gradient_descent(X_train, y_train, alpha=lr, iterations=1000)

    y_val_pred = X_val @ beta
    y_test_pred = X_test @ beta

    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    results.append((lr, val_r2, test_r2))

    print(f"Learning Rate {lr}: Validation R2 = {val_r2:.4f}, Test R2 = {test_r2:.4f}")

    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        best_beta = beta
        best_lr = lr

print("\nBest Learning Rate:", best_lr)
print("Best Validation R2:", best_val_r2)

# Final Test R² using best model
final_test_r2 = r2_score(y_test, X_test @ best_beta)
print("Final Test R2:", final_test_r2)


Shapes: (2800, 6) (700, 6) (1500, 6)
Learning Rate 0.001: Validation R2 = -0.8125, Test R2 = -0.9914
Learning Rate 0.01: Validation R2 = 0.9098, Test R2 = 0.9147
Learning Rate 0.1: Validation R2 = 0.9098, Test R2 = 0.9148
Learning Rate 1: Validation R2 = 0.9098, Test R2 = 0.9148

Best Learning Rate: 0.01
Best Validation R2: 0.9098183094422969
Final Test R2: 0.9147434800538763
